In [1]:
# Install packages
import pandas as pd
import numpy as np

In [143]:
subapps = pd.read_csv('/Users/sambickel-barlow/Desktop/PP434/Portfolio/CC7/HmaSubapplications.csv')
crosswalk = pd.read_csv('/Users/sambickel-barlow/Desktop/PP434/Portfolio/CC7/COUNTY_ZIP_122017.csv')

In [144]:
# Crosswalk from zip code to county code
subapps_df = subapps.merge(crosswalk , left_on='subapplicantZipCode' , right_on='zip')

In [145]:
# Define a function to determine the Hazard type
def hazard(row):
    if (pd.isna(row['primaryHazard']) or row['primaryHazard'] == 'Uncategorized') and (pd.isna(row['secondaryHazard']) or row['secondaryHazard'] == 'Uncategorized'):
        return row['tertiaryHazard']
    elif (pd.isna(row['primaryHazard']) or row['primaryHazard'] == 'Uncategorized') and (not pd.isna(row['secondaryHazard']) and row['secondaryHazard'] != 'Uncategorized'):
        return row['secondaryHazard']
    else:
        return row['primaryHazard']

# Apply the hazard function to create Hazard column
subapps_df['Hazard'] = subapps_df.apply(hazard, axis=1)

In [116]:
# Aggregate claims to the county-Hazard level sum subapplication claims. Remove extraneous columns
FEMA_county_haz = subapps_df[['Hazard' , 'totalSubapplicationAmount' , 'county']].groupby(by = ['county' , 'Hazard']).sum('totalSubapplicationAmount').reset_index()

In [146]:
# Ensure 5 digit county code has leading zeros
FEMA_county_haz['county'] = FEMA_county_haz['county'].apply(lambda x: str(x).zfill(5))

In [136]:
# Crate a dictionary to map Hazard to colors
hazard_colors = {
    'Flooding': '00AAFF',
    'Severe storm': 'FFFF00',
    'Infrastructure failure': '884DFF',
    'Tornado': '4F666A',
    'Fire': 'FF1919',
    'Winter storm': 'A0E6FF',
    'Tropical cyclone (Hurricane/Typhoon)': '00AAFF',
    'Drought': 'CC5500',
    'Dam/Levee break': '00AAFF',
    'Landslide/Debris flow': '644117',
    'Extreme temperature': 'CC5500',
    'Tsunami': '00AAFF'
}


In [137]:
# Create new column mapping hazards to colors
FEMA_county_haz['Hazard_color'] = FEMA_county_haz['Hazard'].apply(lambda x: hazard_colors.get(x, 'FFFFFF'))

In [142]:
# Output to csv
FEMA_county_haz.to_csv('/Users/sambickel-barlow/Desktop/PP434/Portfolio/CC7/FEMA_county_haz.csv' , index=False)